In [81]:
import pandas as pd
from IPython.display import display, HTML
from matplotlib import pyplot as plt
from neo4j import GraphDatabase
from getpass import getpass
from py2neo import Graph
import re
display(HTML("<style>.container { width:100% !important; }</style>"))

In [15]:
# read files
transfer_report_20_21 = pd.read_csv(r'C:\Users\rliam\Desktop\UEI\Projects\School Mobility Graph\xfers_campus_2020_21.csv')
transfer_report_21_22 = pd.read_csv(r'C:\Users\rliam\Desktop\UEI\Projects\School Mobility Graph\xfers_campus_2021_22.csv')
schools_list = pd.read_csv(r'C:\Users\rliam\Desktop\UEI\Projects\School Mobility Graph\Schools_2021_to_2022.csv')

# remove -999 and '.' from TRANSFERS_IN_OR_OUT columns of both datasets
cleaned_transfer_report_20_21 = transfer_report_20_21[(transfer_report_20_21['TRANSFERS_IN_OR_OUT'] != '-999') & (transfer_report_20_21['TRANSFERS_IN_OR_OUT'] != '.')]
cleaned_transfer_report_21_22 = transfer_report_21_22[(transfer_report_21_22['TRANSFERS_IN_OR_OUT'] != '-999') & (transfer_report_21_22['TRANSFERS_IN_OR_OUT'] != '.')]
cleaned_schools_list = schools_list

# reduce columns in schools list to only the 5 below
remove_columns = ['School_Num', 'School_Nam', 'X', 'Y', 'LongLabel']
for column_title in schools_list.keys():
    if column_title not in remove_columns:
        cleaned_schools_list = cleaned_schools_list.drop(columns=column_title)
        
# remove rows in the schools list where school name or school number is NaN
cleaned_schools_list = cleaned_schools_list[cleaned_schools_list['School_Num'].notna()]
cleaned_schools_list = cleaned_schools_list[cleaned_schools_list['School_Nam'].notna()]

# convert school number in cleaned schools list to int type
cleaned_schools_list = cleaned_schools_list.astype({"School_Num":int})

# join school list to each dataset
cleaned_transfer_report_20_21.set_index('CAMPUS_RES_OR_ATTEND').join(cleaned_schools_list.set_index('School_Num'))
cleaned_transfer_report_21_22.set_index('CAMPUS_RES_OR_ATTEND').join(cleaned_schools_list.set_index('School_Num'))

print()

In [16]:
password = getpass('Enter password: ')

Enter password: ········


In [122]:
def delete_all():
    database_connection = GraphDatabase.driver(uri = 'bolt://localhost:7687', auth=('neo4j', password))
    session = database_connection.session()
    query = """
    MATCH (n)
    DELETE n
    """
    session.run(query)
    
    
# Detaches all schools from other schools
def detach_all():
    database_connection = GraphDatabase.driver(uri = 'bolt://localhost:7687', auth=('neo4j', password))
    session = database_connection.session()
    query = "MATCH ()-[r]->() DELETE r;"
    session.run(query)
    

# (Just function name)
def remove_duplicate_relationships():
    ids_to_remove = []
    database_connection = GraphDatabase.driver(uri = 'bolt://localhost:7687', auth=('neo4j', password))
    session = database_connection.session()
    query = """MATCH (s)-[r]->(e)
    WITH s,e,type(r) AS typ, tail(collect(r)) AS coll 
    FOREACH(x IN coll | DELETE x)"""
    session.run(query)

        

In [23]:
create_nodes_queries = []


# Creates school nodes
def give_school_node_query(school_name, school_number, school_x_coord, school_y_coord, school_address):
    query = "CREATE (n:School {Name: \'"
    query += school_name.replace(' ', '_').replace('\'', '_')
    query += "\', Number: "
    query += str(school_number)
    query += ", X: "
    query += str(school_x_coord)
    query += ", Y: "
    query += str(school_y_coord)
    query += ", Address: \'"
    query += school_address
    query += "\'})"
    return query



for index, row in cleaned_schools_list.iterrows():
    query = give_school_node_query(row['School_Nam'], row['School_Num'], row['X'], row['Y'], row['LongLabel'])
    create_nodes_queries.append(query)
    

database_connection = GraphDatabase.driver(uri = 'bolt://localhost:7687', auth=('neo4j', password))
session = database_connection.session()
for query in create_nodes_queries:
    print(query)
    session.run(query)

CREATE (n:School {Name: 'CAYUGA_H_S', Number: 1902001, X: -95.9239602349, Y: 31.9215574497, Address: '17750 N US Highway 287, Tennessee Colony, TX, 75861, USA'})
CREATE (n:School {Name: 'CAYUGA_MIDDLE', Number: 1902041, X: -95.9239602349, Y: 31.9215574497, Address: '17750 N US Highway 287, Tennessee Colony, TX, 75861, USA'})
CREATE (n:School {Name: 'CAYUGA_EL', Number: 1902103, X: -95.9239602349, Y: 31.9215574497, Address: '17750 N US Highway 287, Tennessee Colony, TX, 75861, USA'})
CREATE (n:School {Name: 'ELKHART_H_S', Number: 1903001, X: -95.5806168614, Y: 31.6556229217, Address: '9002 S State Highway 19, Elkhart, TX, 75839, USA'})
CREATE (n:School {Name: 'ELKHART_DAEP', Number: 1903002, X: -95.6383258457, Y: 31.7699921465, Address: '420 W Palestine Ave, Palestine, TX, 75801, USA'})
CREATE (n:School {Name: 'ELKHART_MIDDLE', Number: 1903041, X: -95.5789002476, Y: 31.6266585847, Address: '301 E Parker St, Elkhart, TX, 75839, USA'})
CREATE (n:School {Name: 'ELKHART_EL', Number: 1903101

CREATE (n:School {Name: 'CENTRAL_H_S', Number: 3907001, X: -94.8111114133, Y: 31.4281594441, Address: '7622 US Highway 69 N, Pollok, TX, 75969, USA'})
CREATE (n:School {Name: 'STUBBLEFIELD_LRN_CTR', Number: 3907004, X: -94.7603589861, Y: 31.3345121962, Address: '208 N John Redditt Dr, Lufkin, TX, 75904, USA'})
CREATE (n:School {Name: 'CENTRAL_J_H', Number: 3907041, X: -94.8111114133, Y: 31.4281594441, Address: '7622 US Highway 69 N, Pollok, TX, 75969, USA'})
CREATE (n:School {Name: 'CENTRAL_EL', Number: 3907102, X: -94.8269532463, Y: 31.4342334139, Address: '8578 US Highway 69 N, Pollok, TX, 75969, USA'})
CREATE (n:School {Name: 'ROCKPORT-FULTON_H_S', Number: 4901001, X: -97.0450475835, Y: 28.0409161655, Address: '1801 Omohundro St, Rockport, TX, 78382, USA'})
CREATE (n:School {Name: 'ROCKPORT-FULTON_MIDDLE', Number: 4901041, X: -97.0461849608, Y: 28.0460127659, Address: '1701 Colorado Ave, Rockport, TX, 78382, USA'})
CREATE (n:School {Name: 'FULTON_LEARNING_CENTER', Number: 4901101, X

CREATE (n:School {Name: 'RED_ROCK_EL', Number: 11901107, X: -97.4517609074, Y: 29.9597036512, Address: '2401 FM 20, Red Rock, TX, 78662, USA'})
CREATE (n:School {Name: 'CEDAR_CREEK_INT', Number: 11901108, X: -97.4794979661, Y: 30.0993089844, Address: '151 Voss Pkwy, Cedar Creek, TX, 78612, USA'})
CREATE (n:School {Name: 'BLUEBONNET_EL', Number: 11901109, X: -97.428912007, Y: 30.1317801001, Address: '416 FM 1209, Bastrop, TX, 78602, USA'})
CREATE (n:School {Name: 'LOST_PINES_EL', Number: 11901110, X: -97.2808182786, Y: 30.1772962232, Address: '151 Tiger Woods Dr, Bastrop, TX, 78602, USA'})
CREATE (n:School {Name: 'ELGIN_H_S', Number: 11902001, X: -97.4034636269, Y: 30.3657740868, Address: '14000 County Line Rd, Elgin, TX, 78621, USA'})
CREATE (n:School {Name: 'BASTROP_COUNTY_JUVENILE_BOOT_CAMP', Number: 11902003, X: -97.3776769533, Y: 30.3526603479, Address: '1002 N Avenue C, Elgin, TX, 78621, USA'})
CREATE (n:School {Name: 'PHOENIX_H_S', Number: 11902004, X: -97.4034636269, Y: 30.36577

CREATE (n:School {Name: 'CHARTER_OAK_EL', Number: 14903120, X: -97.4407624255, Y: 31.0969952266, Address: '8402 Poison Oak Rd, Temple, TX, 76502, USA'})
CREATE (n:School {Name: 'HOLLAND_H_S', Number: 14905001, X: -97.4097523358, Y: 30.8807457153, Address: '100 S Rose Ln, Holland, TX, 76534, USA'})
CREATE (n:School {Name: 'BELL_COUNTY_JJAEP', Number: 14905003, X: -99.33305009, Y: 31.462733046, Address: 'TX, USA'})
CREATE (n:School {Name: 'BELL_COUNTY_DAEP', Number: 14905004, X: -97.3465189046, Y: 30.9763523149, Address: '706 E Rio Poco, Little River Academy, TX, 76554, USA'})
CREATE (n:School {Name: 'HOLLAND_MIDDLE', Number: 14905041, X: -97.4099041796, Y: 30.8777035911, Address: '302 Hackberry Rd, Holland, TX, 76534, USA'})
CREATE (n:School {Name: 'HOLLAND_EL', Number: 14905101, X: -97.4092971409, Y: 30.8794217912, Address: '503 Crockett St, Holland, TX, 76534, USA'})
CREATE (n:School {Name: 'KILLEEN_H_S', Number: 14906001, X: -97.6995985292, Y: 31.1124181799, Address: '500 N 38th St, 

CREATE (n:School {Name: 'ROGERS_H_S', Number: 14907001, X: -97.2254456905, Y: 30.9332162231, Address: '210 Alvin Ailey St, Rogers, TX, 76569, USA'})
CREATE (n:School {Name: 'ROGERS_MIDDLE', Number: 14907041, X: -97.2305456927, Y: 30.9296129874, Address: '114 Eagle Dr, Rogers, TX, 76569, USA'})
CREATE (n:School {Name: 'BELL_COUNTY_JJAEP', Number: 14907042, X: -97.2312713402, Y: 30.9297294872, Address: '1 Eagle Dr, Rogers, TX, 76569, USA'})
CREATE (n:School {Name: 'BELL_COUNTY_DAEP', Number: 14907043, X: -97.2312713402, Y: 30.9297294872, Address: '1 Eagle Dr, Rogers, TX, 76569, USA'})
CREATE (n:School {Name: 'ROGERS_EL', Number: 14907102, X: -97.2234573947, Y: 30.924090981, Address: '802 Rogers Cemetery Rd, Rogers, TX, 76569, USA'})
CREATE (n:School {Name: 'SALADO_H_S', Number: 14908001, X: -97.5405217416, Y: 30.9649528009, Address: '1880 Williams Rd, Salado, TX, 76571, USA'})
CREATE (n:School {Name: 'BELL_CO_JJAEP', Number: 14908002, X: -97.6766588528, Y: 31.115847929, Address: '4800 E 

...

CREATE (n:School {Name: 'SHEPPARD_AFB_EL', Number: 243905123, X: -98.504825958, Y: 33.9620412283, Address: '301 Anderson Dr, Sheppard Afb, TX, 76311, USA'})
CREATE (n:School {Name: 'BOOKER_T_WASHINGTON_EL', Number: 243905125, X: -98.4677403599, Y: 33.9138975292, Address: '1300 Harding St, Wichita Falls, TX, 76301, USA'})
CREATE (n:School {Name: 'WEST_FOUNDATION_EL', Number: 243905126, X: -98.5746517026, Y: 33.8766391733, Address: '5220 Lake Wellington Pkwy, Wichita Falls, TX, 76310, USA'})
CREATE (n:School {Name: 'SCOTLAND_PARK_EL', Number: 243905128, X: -98.5087739669, Y: 33.9197949525, Address: '1415 N 5th St, Wichita Falls, TX, 76306, USA'})
CREATE (n:School {Name: 'SOUTHERN_HILLS_EL', Number: 243905129, X: -98.4856438537, Y: 33.8735108794, Address: '3920 Armory Rd, Wichita Falls, TX, 76302, USA'})
CREATE (n:School {Name: 'FARRIS_EARLY_CHILDHOOD_CTR', Number: 243905130, X: -98.4979362499, Y: 33.9252843447, Address: '710 Burkburnett Rd, Wichita Falls, TX, 76306, USA'})
CREATE (n:Scho

CREATE (n:School {Name: 'STEP_-_JJAEP', Number: 246906002, X: -97.6594922122, Y: 30.6179371898, Address: '200 Wilco Way, Georgetown, TX, 78626, USA'})
CREATE (n:School {Name: 'STEP_--DETENTION', Number: 246906003, X: -97.6594922122, Y: 30.6179371898, Address: '200 Wilco Way, Georgetown, TX, 78626, USA'})
CREATE (n:School {Name: 'HUTTO_NINTH_GRADE_CENTER', Number: 246906004, X: -97.5453567587, Y: 30.5812158625, Address: '301 Destiny Ln, Hutto, TX, 78634, USA'})
CREATE (n:School {Name: 'STEP_-_CORE', Number: 246906005, X: -97.6594922122, Y: 30.6179371898, Address: '200 Wilco Way, Georgetown, TX, 78626, USA'})
CREATE (n:School {Name: 'HUTTO_MIDDLE', Number: 246906041, X: -97.5532584626, Y: 30.5526410325, Address: '1005 Exchange Blvd, Hutto, TX, 78634, USA'})
CREATE (n:School {Name: 'FARLEY_MIDDLE', Number: 246906042, X: -97.5430062216, Y: 30.5161612297, Address: '303 Cr 137, Hutto, TX, 78634, USA'})
CREATE (n:School {Name: 'HUTTO_EL', Number: 246906101, X: -97.5393222075, Y: 30.5505683555

CREATE (n:School {Name: 'KATHY_CARAWAY_EL', Number: 246909110, X: -97.7681840387, Y: 30.4188534526, Address: '11104 Oak View Dr, Austin, TX, 78759, USA'})
CREATE (n:School {Name: 'VIC_ROBERTSON_EL', Number: 246909111, X: -97.6634053875, Y: 30.5240791526, Address: '1415 Bayland St, Round Rock, TX, 78664, USA'})
CREATE (n:School {Name: 'PURPLE_SAGE_EL', Number: 246909113, X: -97.8202973482, Y: 30.457108333, Address: '11801 Tanglebriar Trl, Austin, TX, 78750, USA'})
CREATE (n:School {Name: 'BRUSHY_CREEK_EL', Number: 246909114, X: -97.7375415609, Y: 30.5116140472, Address: '3800 Stonebridge, Round Rock, TX, 78681, USA'})
CREATE (n:School {Name: 'WELLS_BRANCH_EL', Number: 246909115, X: -97.6865609989, Y: 30.4457406781, Address: '14650 Merriltown Dr, Austin, TX, 78728, USA'})
CREATE (n:School {Name: 'LAUREL_MOUNTAIN_EL', Number: 246909116, X: -97.7814753064, Y: 30.4137767434, Address: '10111 D K Ranch Rd, Austin, TX, 78759, USA'})
CREATE (n:School {Name: 'POND_SPRINGS_EL', Number: 246909117,

CREATE (n:School {Name: 'STEINER_RANCH_EL', Number: 246913107, X: -97.8849789863, Y: 30.379619105, Address: '4001 N Quinlan Park Rd, Austin, TX, 78732, USA'})
CREATE (n:School {Name: 'PAULINE_NAUMANN_EL', Number: 246913108, X: -97.8301267223, Y: 30.4767174602, Address: '1201 Brighton Bend Ln, Cedar Park, TX, 78613, USA'})
CREATE (n:School {Name: 'BAGDAD_EL', Number: 246913109, X: -97.8797884089, Y: 30.5790770103, Address: '800 Deercreek Ln, Leander, TX, 78641, USA'})
CREATE (n:School {Name: 'CHARLOTTE_COX_EL', Number: 246913110, X: -97.8055344698, Y: 30.5035208587, Address: '1001 Brushy Creek Rd, Cedar Park, TX, 78613, USA'})
CREATE (n:School {Name: 'LAURA_WELCH_BUSH_EL', Number: 246913111, X: -97.9012325024, Y: 30.3655962648, Address: '12600 Country Trl, Austin, TX, 78732, USA'})
CREATE (n:School {Name: 'PATRICIA_KNOWLES_EL', Number: 246913112, X: -97.8652738025, Y: 30.5311066241, Address: '2101 Cougar Country Dr, Cedar Park, TX, 78613, USA'})
CREATE (n:School {Name: 'DEER_CREEK_EL', 

CREATE (n:School {Name: 'PARADISE_MIDDLE', Number: 249906041, X: -97.6918374736, Y: 33.1438709857, Address: '338 School House Rd, Paradise, TX, 76073, USA'})
CREATE (n:School {Name: 'PARADISE_INT', Number: 249906050, X: -97.6918374736, Y: 33.1438709857, Address: '338 School House Rd, Paradise, TX, 76073, USA'})
CREATE (n:School {Name: 'PARADISE_EL', Number: 249906101, X: -97.6918374736, Y: 33.1438709857, Address: '338 School House Rd, Paradise, TX, 76073, USA'})
CREATE (n:School {Name: 'SLIDELL_SCHOOLS', Number: 249908001, X: -97.3952329083, Y: 33.3590889989, Address: '1 Greyhound Ln, Decatur, TX, 76234, USA'})
CREATE (n:School {Name: 'HAWKINS_H_S_/_MIDDLE', Number: 250902001, X: -95.2078274429, Y: 32.5819549861, Address: '179 Hawk Dr, Hawkins, TX, 75765, USA'})
CREATE (n:School {Name: 'HAWKINS_EL', Number: 250902101, X: -95.2078274429, Y: 32.5819549861, Address: '179 Hawk Dr, Hawkins, TX, 75765, USA'})
CREATE (n:School {Name: 'MINEOLA_H_S', Number: 250903001, X: -95.4987079815, Y: 32.

In [24]:
create_relationships_queries = []

# create relationship query based on args
def create_relationship(school1_num, school2_num, num_transfers):
    
    insertion_query = """
    MATCH (a:School), (b:School)
    WHERE a.Number = {} AND b.Number = {}
    CREATE (a)-[r:TRANSFERRED_{}_TO]->(b) 
    RETURN type(r)
    """.format(int(school1_num), int(school2_num), num_transfers)
    return insertion_query


# create relationships between schools with a student transfer relationship
for index, row in cleaned_transfer_report_21_22.iterrows():
    if row['REPORT_TYPE'] == 'Transfers In From':
        query = create_relationship(row['CAMPUS_RES_OR_ATTEND'], row['REPORT_CAMPUS'], row['TRANSFERS_IN_OR_OUT'])
        create_relationships_queries.append(query)
    else:
        query = create_relationship(row['REPORT_CAMPUS'], row['CAMPUS_RES_OR_ATTEND'], row['TRANSFERS_IN_OR_OUT'])
        create_relationships_queries.append(query)
        

# connect to database and run queries
database_connection = GraphDatabase.driver(uri = 'bolt://localhost:7687', auth=('neo4j', password))
session = database_connection.session()
for query in create_relationships_queries:
    session.run(query)
    
    

remove_duplicate_relationships()